# AutoFlight Prosperity I - Mission Feasibility

Chinese air taxi startup Autoflight (https://www.autoflight.com/) announces what it claims is the world’s longest eVTOL flight to date. Its electric flying cab, ‘Prosperity I’, allegedly covered a distance of 250 kilometres without stopping to charge. The flying taxi is scheduled for certification in 2025. <br>

The mission was recorded in the following youtube video: <br>
https://www.youtube.com/watch?v=2Xf1_lTcAN0

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime as datetime

In [7]:
class mission_segment():
    def __init__(self, name, t1, t2, dist, alt):
        """
        name = segment name
        t1 = start time (datetime object)
        t2 = end time (datetime object)
        dist = distance list ([start,end]) in km
        alt = altitude list ([start,end]) in m
        """
        self.name = name
        self.t1 = t1
        self.t2 = t2
        self.dt = (t2 - t1).total_seconds()
        self.dist = dist
        self.alt = alt
        self._calc_info()
    def _calc_info(self):
        if self.name == 'Takeoff':
            # average vertical speed (m/s)
            self.vy = (self.alt[1]-self.alt[0])/self.dt
        elif self.name == 'Forward Transition':
            # horizontal acceleration (m/s2) (manually input v_cruise=44)
            v_cruise = 44
            self.ax = (v_cruise)/self.dt
        elif self.name == 'Climb':
            # average vertical speed (m/s)
            self.vy = (self.alt[1]-self.alt[0])/self.dt
        elif self.name == 'Fixed Wing Cruise':
            # cruise speed (km/h)
            self.vx = (self.dist[1]-self.dist[0])/(self.dt/3600)
        elif self.name == 'Descend':
            # average vertical speed (m/s)
            self.vy = (self.alt[1]-self.alt[0])/self.dt
        elif self.name == 'Backward Transition':
            # horizontal acceleration (m/s2) (manually input v_cruise=44)
            v_cruise = 44
            self.ax = (-v_cruise)/self.dt
        elif self.name == 'Landing':
            # average vertical speed (m/s)
            self.vy = (self.alt[1]-self.alt[0])/self.dt

#### From the above youtube video, the following data were noted.

In [8]:
mission_segment_list = []

mission_segment_list.append(mission_segment(name = 'Takeoff',
                                            t1   = datetime(2023,2,23,0,0,0),
                                            t2   = datetime(2023,2,23,0,0,40),
                                            dist = [0.0, 0.0032],
                                            alt  = [0.0, 80.0]))

mission_segment_list.append(mission_segment(name = 'Forward Transition',
                                            t1   = datetime(2023,2,23,0,0,40),
                                            t2   = datetime(2023,2,23,0,1,0),
                                            dist = [0.0032, 0.488],
                                            alt  = [80.0, 80.0]))

mission_segment_list.append(mission_segment(name = 'Climb',
                                            t1   = datetime(2023,2,23,0,1,0),
                                            t2   = datetime(2023,2,23,0,2,30),
                                            dist = [0.488, 4.0],
                                            alt  = [80.0, 120.0]))

mission_segment_list.append(mission_segment(name = 'Fixed Wing Cruise',
                                            t1   = datetime(2023,2,23,0,2,30),
                                            t2   = datetime(2023,2,23,1,34,0),
                                            dist = [4.0, 245.7],
                                            alt  = [120.0, 120.0]))

mission_segment_list.append(mission_segment(name = 'Descend',
                                            t1   = datetime(2023,2,23,1,34,0),
                                            t2   = datetime(2023,2,23,1,35,27),
                                            dist = [245.7, 249.4],
                                            alt  = [120.0, 80.0]))

mission_segment_list.append(mission_segment(name = 'Backward Transition',
                                            t1   = datetime(2023,2,23,1,35,38),
                                            t2   = datetime(2023,2,23,1,36,13),
                                            dist = [249.4, 250.2],
                                            alt  = [80.0, 80.0]))

mission_segment_list.append(mission_segment(name = 'Landing',
                                            t1   = datetime(2023,2,23,1,36,13),
                                            t2   = datetime(2023,2,23,1,37,13),
                                            dist = [250.2, 250.3],
                                            alt  = [80.0, 0.0]))

#### The above mission is visualized as follows.

In [9]:
plot_list = []

dx = [10,23,9,0,-13,-24,-11]
dy = [0,0,0,5,0,0,0]

for i, seg in enumerate(mission_segment_list):
    go_scatter = go.Scatter(mode = 'lines+markers',
                            x = seg.dist,
                            y = seg.alt,
                            name = seg.name,
                            line = dict(color='blue'),
                            text = seg.name,
                            hovertemplate = '<extra></extra>dist = %{x} km<br>'+'alt   = %{y}.0 m')
    plot_list.append(go_scatter)
    go_text = go.Scatter(mode = 'text',
                     x = [0.5*(seg.dist[0]+seg.dist[1])+dx[i]],
                     y = [0.5*(seg.alt[0]+seg.alt[1])+dy[i]],
                     text=f'<b>{seg.name}</b>',
                     hoverinfo='none')
    plot_list.append(go_text)

# Takeoff segment
seg = mission_segment_list[0]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1])+15],
                 y = [0.5*(seg.alt[0]+seg.alt[1])-15],
                 text=f'{seg.vy} m/s<br>vertical climb<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)

# Forward transition segment
seg = mission_segment_list[1]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1]+150)],
                 y = [0.5*(seg.alt[0]+seg.alt[1])],
                 text=f'{seg.ax:.2f} m/s2<br>horizontal acceleration<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)

# Climb
seg = mission_segment_list[2]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1])+35],
                 y = [0.5*(seg.alt[0]+seg.alt[1])],
                 text=f'Vy = {seg.vy:.2f} m/s<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)

# Cruise segment
seg = mission_segment_list[3]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1])],
                 y = [0.5*(seg.alt[0]+seg.alt[1])-10],
                 text=f'{seg.dist[1]-seg.dist[0]} km cruise at {seg.vx:.2f} km/h<br>for {seg.dt/3600} hours',
                 hoverinfo='none')
plot_list.append(go_text)

# Descend
seg = mission_segment_list[4]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1])-40],
                 y = [0.5*(seg.alt[0]+seg.alt[1])],
                 text=f'Vy = {seg.vy:.2f} m/s<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)

# Backward transition segment
seg = mission_segment_list[5]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1]-150)],
                 y = [0.5*(seg.alt[0]+seg.alt[1])],
                 text=f'{seg.ax:.2f} m/s2<br>horizontal acceleration<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)

# Landing segment
seg = mission_segment_list[6]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1])-20],
                 y = [0.5*(seg.alt[0]+seg.alt[1])-15],
                 text=f'{seg.vy:.2f} m/s<br>vertical descend<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)   

# Summary
duration = (mission_segment_list[-1].t2-mission_segment_list[0].t1).total_seconds()
hours = duration//3600
minutes = (duration % 3600)//60
seconds = duration % 60
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(mission_segment_list[3].dist[0]+mission_segment_list[3].dist[1])],
                 y = [0.5*(mission_segment_list[3].alt[0]+mission_segment_list[3].alt[1])-80],
                 text=f'<b>Mission Summary</b><br>Date: {mission_segment_list[0].t1:%b %d, %Y}<br>Duration: {hours}hour, {minutes} mins, {seconds} secs',
                 hoverinfo='none')
plot_list.append(go_text) 

layout = go.Layout(title=go.layout.Title(text="Simplified Mission Profile",x=0.5),
                   xaxis={'title':'Distance [km]','range':[-11,261]},
                   yaxis={'title':'Altitude [m]','range':[-10,140]})

fig = go.Figure(data=plot_list, layout=layout)
fig.update_layout(showlegend=False)
fig.show()

## Prosperity I - Specifications and Performance Approximations

### General

The Prosperity I's specifications are listed below.<br>

<b>Propulsion</b>: Fully electric Lift+Cruise<br>
<b>Powertrain</b>: 10 lifting rotors + 3 propellers<br>
<b>Wingspan</b>: 12.8 m<br>
<b>Height</b>: 3.3 m<br><br>

The Prosperity I's performance is listed below.<br>

<b>Max range</b>: 241.402 km+<br>
<b>Cruise speed</b>: 209.215 km/h+:<br>
<b>Max payload</b>: 408.233 kg+<br>
<b>Hover noise</b>: ~70 dBA<br><br>

<b>source</b> = https://www.autoflight.com/en/products/

In [5]:
no_of_crews = 4 # people
MTOW = 1500 # kg
payload = no_of_crews * 90 # kg (assuming 1 person = 90 kg)
PWR = payload / MTOW # payload-to-weight-ratio
print(f'Payload = {payload} kg\nPWR = {PWR}')

Payload = 360 kg
PWR = 0.24


### Powertrain

In [6]:
prop_diam = 2.49 # propeller diameter (m)
no_of_lift_props = 10
no_of_push_props = 3
lift_prop_area = no_of_lift_props * np.pi * (prop_diam/2)**2
g = 9.81 # m/s2 (gravitational acceleration)
disc_loading = MTOW*g/lift_prop_area
print(f'Disc loading = {disc_loading:.2f} N/m2')

Disc loading = 302.18 N/m2


### Power and Energy Requirement

Based on the mission profile, the power and energy required can then be calculated.<br>

The power required for the vertical take-off and landing segments are:

\begin{equation*}
P_{VTOL} = \sqrt \frac{T^3}{2\rho_{sea}A_{hover}}
\tag{1}
\end{equation*}<br>

where $T$ is the hovering thrust, $\rho$ is the air density, and $A_{hover}$ is the disk actuator area.

While the power required for the cruise segment:

\begin{equation*}
P_{cruise} = \frac{D_{cruise}v_{cruise}}{\eta_{p}}
\tag{2}
\end{equation*}<br>

where $D_{cruise}$ is the total drag during cruise, $v_{cruise}$ is the cruise speed, and $\eta_{p}$ is the propeller efficiency.

<b>source</b>: Coleman, C.P., <em>A Survey of Theoretical and Experimental Coaxial Rotor Aerodynamic Research</em>; NASA Ames Research Center: Mountain View, CA, USA, 1997; Technical Paper 3675.


In [7]:
T = MTOW*g
rho = 1.225 # kg/m3
A = lift_prop_area
P_hover = np.sqrt((T**3)/(2*rho*A)) / 1000 # kW
hover_time = mission_segment_list[0].dt + mission_segment_list[-1].dt
E_hover = P_hover * hover_time / 3600 #kWh
print(f'Required hover power = {P_hover:.2f} kW')
print(f'Hover time = {hover_time} s')
print(f'Required hover energy = {E_hover:.2f} kWh')

Required hover power = 163.42 kW
Hover time = 100.0 s
Required hover energy = 4.54 kWh


### Range equation for electric aircraft

The electric energy consumption of the aircraft is: <br><br>
\begin{equation*}
\tilde{E}m_{batt} \eta_{t} = W \frac{L}{D}
\tag{3}
\end{equation*}<br>
where $\tilde{E}$ is the battery energy density, $m_{batt}$ is the battery mass, $\eta_{t}$ is the consumption efficiency, $W$ is the mass, $L$ is the lift force, $D$ is the drag force, and $R$ is the range. Rewriting the equation for range, it yields:<br><br>
\begin{equation*}
R = 0.75 \tilde{E} \frac{m_{batt}}{m} \frac{1}{g} \frac{L}{D}
\tag{4}
\end{equation*}<br>
where $m$ is the aircraft mass, $g$ is the gravity acceleration, and $\eta_{t}$ is assumed to be 0.75 (conservative).

<b>source</b>: Raymer, D.P., <em>Aircraft Design: A Conceptual Approach,</em> 6th ed.; AIAA: Reston, VA. USA, 2018.

In [34]:
# For doing reversed engineering purpose: mtow is set as a function
def calc_mtow(R, E_tilde, m_batt, L_by_D):
    """
    R = cruising range (km)
    E_tilde = Battery Energy Density (Wh/kg)
    m_batt = battery mass (kg)
    L_by_D = Lift-to-drag ratio
    """
    # Convert to SI
    R = R * 1000 # m
    E_tilde = E_tilde * 3600 # Ws/kg
    g = 9.81 # m/s^2
    
    mtow = 0.75 * E_tilde * m_batt * L_by_D / g / R
    
    return mtow

### Aerodynamics
A drag force is calculated using the following equations.<br><br>

\begin{equation*}
D = \frac{1}{2} \rho v^2 S C_{D}
\tag{5}
\end{equation*}<br>

\begin{equation*}
C_{D} = C_{D_{p}} + C_{D_{i}}
\tag{6}
\end{equation*}<br>

where $\rho$ is the air density, $v$ is the aircraft velocity, $S$ is the wing area, $C_{D_{p}}$ is the parasite drag, and $C_{D_{i}}$ is the induced drag.<br>

The induced drag coefficient value is expressed as the following equation, assuming an ideal elliptical lift distribution.<br><br>

\begin{equation*}
C_{D_{i}} = \frac{C_{L}^2}{\pi e AR}
\tag{7}
\end{equation*}<br>

\begin{equation*}
AR = \frac{b^2}{S}
\tag{8}
\end{equation*}<br>

where $C_{L}$ is the lift coefficient, $e$ is the Oswald span efficiency coefficient, $AR$ is the aspect ratio, and $b$ is the wing span.<br>

When cruising, the lift coefficient is as follows.<br><br>

\begin{equation*}
C_{L} = \frac{W}{\frac{1}{2} \rho v^2 S}
\tag{9}
\end{equation*}<br>


\begin{equation*}
e = 1.78(1-0.045AR^{0.68})-0.64
\tag{10}
\end{equation*}<br>

Substituting $C_{L}$ in Eq(9) into Eq(7) yields:<br><br>

\begin{equation*}
C_{D_{i}} = \frac{1}{\pi e AR}\big(\frac{W}{\frac{1}{2}\rho v^2S}\big)^2
\tag{11}
\end{equation*}<br>

Substituting $C_{D_{i}}$ in the Eq(11) into Eq(6):<br><br>

\begin{equation*}
C_{D} = C_{D_{p}} + \frac{1}{\pi e AR}\big(\frac{W}{\frac{1}{2}\rho v^2S}\big)^2
\tag{12}
\end{equation*}<br>

Substituting $C_{D}$ in Eq(12) into Eq(5):<br><br>

\begin{equation*}
D = \frac{1}{2}\rho v^2S \big( C_{D_{p}} + \frac{1}{\pi e AR}\big(\frac{W}{\frac{1}{2}\rho v^2S}\big)^2 \big)
\tag{13}
\end{equation*}<br>

In order to calculate the total drag, it is necessary to calculate the parasite drag. The component build-up method is used to calculate the parasite drag. The parasite drag at subsonic speed is expressed by the following equation.<br><br>

\begin{equation*}
C_{D_{p}} = \frac{\Sigma\big(C_{f}FFQS_{wet}\big)}{S_{ref}} + C_{D_{misc}} + C_{D_{L\&P}} 
\tag{14}
\end{equation*}<br>

where $C_{f}$ is the surface friction coefficient, $FF$ is the form factor, $Q$ is the interference coefficient, $S_{wet}$ is the wetted area, $S_{ref}$ is the reference area, $C_{D_{misc}}$ is the miscellaneous drag, and $C_{D_{L\&P}}$ is the leakages and protuberances drag coefficient.<br>

The surface friction coefficient of the plate is expressed by the following equation for laminar and turbulent flows.<br>

Laminar:
\begin{equation*}
C_{f} = \frac{1.328}{\sqrt{R}}
\tag{15}
\end{equation*}<br>

Turbulent:
\begin{equation*}
C_{f} = \frac{0.455}{(log_{10}R)^{2.58}(1+0.144M^2)^{0.65}}
\tag{16}
\end{equation*}<br>

where $R$ is the Reynolds number and $M$ is the Mach number.

<b>source</b>: Raymer, D.P., <em>Aircraft Design: A Conceptual Approach,</em> 6th ed.; AIAA: Reston, VA. USA, 2018.

In [24]:
wing_chord = 1
wing_span = 12.8
wing_area = wing_chord * wing_span
AR = wing_span**2 / wing_area
e = 1.78 * (1 - 0.045 * AR**0.68) - 0.64

In [ ]:
#Induced drag
mtow = 1500
rho = 1.225 # kg/m3
v_cruise = 207 # km/h
v_cruise = v_cruise * 1000 / 3600 # m/s
W = mtow * 9.81
CL_cruise = W/(0.5*rho*v_cruise**2*wing_area)
CD_i = CL_cruise**2/(np.pi*e*AR)

In [25]:
# Parasite drag
rho = 1.17
g = 9.81
K = 0.15
CD_p = K * (2*mtow*g/wing_area/rho)**2 / v_cruise**4

In [26]:
# Total drag
CD = CD_i + CD_p
CD

0.06466498108077257

### Range calculation

In [35]:
# Battery energy density (Wh/kg)
E_tilde = [200, 220, 240, 260, 280, 300]

# Range
Range = [50, 100, 150, 200, 250]

# M_battery is assumed to be 30% of mtow
m_batt = 450

# Lift-to-drag ratio
L_by_D = CL_cruise/CD

mtow = calc_mtow(Range[0], E_tilde[0], m_batt, L_by_D)

In [36]:
mtow

4349.172380841927